# Potencial en forma de V

El potencial en forma de V es una función que tiene parámetros que determina la pendiente del potencial. Este tipo de potencial es característico de sistemas físicos donde la energía potencial crece linealmente con la distancia desde un punto de referencia, lo que se observa en diferentes contextos de la mecánica cuántica y la física clásica.

## Características del potencial en forma de V
1. **Simetría**: El potencial es simétrico respecto al eje $x = 0$, lo que significa que la fuerza resultante es igual en magnitud pero opuesta en dirección para posiciones positivas y negativas.
2. **Crecimiento lineal**: A diferencia de un pozo de potencial cuadrático (como el oscilador armónico), este potencial crece de manera lineal a medida que nos alejamos del origen.
3. **Energía y confinamiento**: Este potencial no posee paredes infinitas dentro de un intervalo finito como un **pozo de potencial infinito**, pero si se considera dentro de un dominio restringido $x_0 < x < x_{N+1}$, el sistema se comporta como una partícula confinada.

## Importancia en física cuántica
El potencial en forma de V se usa en varios modelos físicos, tales como:
- La aproximación al potencial de partículas en campos externos lineales.
- Modelos de confinamiento en física de partículas.
- Aproximaciones a sistemas con fuerzas elásticas no lineales.

En mecánica cuántica, resolver la ecuación de Schrödinger para este potencial implica encontrar las funciones de onda $\psi(x)$ y los valores de energía permitidos $E_n$. Una manera numérica de abordar esta solución es usando el **método de Numerov**, que permite obtener soluciones precisas a ecuaciones diferenciales de segundo orden como la ecuación de Schrödinger independiente del tiempo.

Este potencial es interesante porque, a diferencia de otros modelos con soluciones analíticas, su tratamiento suele requerir métodos numéricos. La función de onda $\psi(x)$ típicamente se encuentra confinada en una región finita y decrece rápidamente fuera de los límites definidos por la energía máxima $E_m$.

## Representación gráfica
En la representación gráfica del potencial en forma de V, se observa que:
- Para $x = 0$, el potencial es cero.
- A medida que $x$ se aleja del origen, el potencial crece proporcionalmente a $|x|$.
- Si se imponen barreras de potencial infinito en $x_0$ y $x_{N+1}$, la función de onda deberá anularse en estos puntos, similar a una partícula confinada en un pozo de potencial.

Este modelo sirve como un buen ejercicio para estudiar métodos numéricos aplicados a problemas físicos y para comprender el comportamiento de sistemas cuánticos en potenciales no triviales.


# Método de Numerov

El método de Numerov es un algoritmo numérico para resolver ecuaciones
diferenciales de segundo orden de la forma:

$$
\frac{d^2 y(x)}{dx^2} = f(x)\, y(x)
$$

En mecánica cuántica se aplica a la ecuación de Schrödinger independiente del tiempo:

$$
-\frac{\hbar^2}{2m}\frac{d^2 \psi}{dx^2} + V(x)\psi = E\psi
$$

que puede escribirse como:

$$
\psi''(x) = \frac{2m}{\hbar^2}\left[V(x)-E\right]\psi(x)
$$

Numerov permite obtener numéricamente:

- Las funciones de onda $\psi_n(x)$  
- Los niveles de energía $E_n$

para cualquier potencial $V(x)$ definido en una malla discreta.

El potencial considerado es:

$$
V(x) =
\begin{cases}
\infty, & x \leq x_0 \\
V_0 x^2, & x_0 < x < x_{N+1} \\
\infty, & x \geq x_{N+1}
\end{cases}
$$

donde:
- $ V_0 $ controla la curvatura del potencial.
- $ x_0 $ y $ x_{N+1} $ son los límites donde el potencial se hace infinito.
- Este corresponde al oscilador armónico cuántico confinado.

El potencial considerado es:

$$
V(x) =
\begin{cases}
\infty, & x \leq x_0 \\
V_0 x^4, & x_0 < x < x_{N+1} \\
\infty, & x \geq x_{N+1}
\end{cases}
$$

donde:
- $ V_0 $ controla la intensidad del confinamiento.
- Este potencial es más "empinado" que el armónico.
- Produce niveles de energía no equiespaciados.

El potencial considerado es:

$$
V(x) =
\begin{cases}
\infty, & x \leq x_0 \\
V_0 (x^2 - a^2)^2, & x_0 < x < x_{N+1} \\
\infty, & x \geq x_{N+1}
\end{cases}
$$

donde:
- $V_0$ controla la profundidad.
- $a$ determina la separación entre pozos.
- Es un modelo clásico para estudiar efecto túnel.

-----------------------------------------------------------------------------------------------------------------------


## Oscilador cuántico

In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh

def oscilador_cuantico_numerov():
    Rmin = -10.0
    Rmax = 10.0
    N = 50
    Nf = 1000
    
    print("Los 6 primeros autovalores:")
    
    while N <= Nf:
        # Definición de la malla espacial (h)
        h = (Rmax - Rmin) / N
        # En Fortran tu malla va desde Rmin+h hasta Rmin+N*h
        x = np.linspace(Rmin + h, Rmin + N*h, N)
        
        # Matriz del Potencial V(x) = 0.5 * x^2
        V_diag = 0.5 * x**2
        V = sp.diags([V_diag], [0])
        
        # Matriz de Energía Cinética (T)
        diag_T = np.full(N, 1.0 / h**2)
        off_diag_T = np.full(N - 1, -1.0 / (2.0 * h**2))
        T = sp.diags([off_diag_T, diag_T, off_diag_T], [-1, 0, 1])
        
        # Matriz de Promedios de Numerov (B)
        diag_B = np.full(N, 10.0 / 12.0)
        off_diag_B = np.full(N - 1, 1.0 / 12.0)
        B = sp.diags([off_diag_B, diag_B, off_diag_B], [-1, 0, 1])
        
        # Matriz Hamiltoniana Generalizada (A = T + B*V)
        A = T + B.dot(V)
        
        # Resolvemos el problema de autovalores generalizado A * psi = E * B * psi
        # which='SA' busca los autovalores algebraicos más pequeños (Smallest Algebraic)
        autovalores, autovectores = eigsh(A, k=6, M=B, which='SA')
        
        # Resultados (N y los primeros 6 autovalores)
        # Los autovalores exactos teóricos son: 0.5, 1.5, 2.5, 3.5, 4.5, 5.5
        formato_impresion = f"{N:4d}  " + "  ".join([f"{val:9.6f}" for val in autovalores])
        print(formato_impresion)
        
        if N >= Nf / 2:
            with open("data_oscilador_numerov.txt", "w") as f:
                # Cabecera
                f.write(f"# {N//2:4d}      " + " ".join([f"{val:10.6f}" for val in autovalores]) + "\n")
                
                # Cálculo de la norma para la densidad de probabilidad |psi|^2
                normas = np.sum(autovectores**2, axis=0) * h
                densidades = (autovectores**2) / normas
                
                # Guardar malla y densidades
                for i in range(N):
                    linea = " ".join([f"{densidades[i, j]:10.5f}" for j in range(6)])
                    f.write(f"{x[i]:10.5f} {linea}\n")
                    
        N *= 2
        
    print("\nAutovectores guardados en data_oscilador_numerov.txt")

if __name__ == '__main__':
    oscilador_cuantico_numerov()

Los 6 primeros autovalores:
  50   0.506319   1.517568   2.473162   3.480949   4.485612   5.513334
 100   0.497966   1.495923   2.497637   3.496249   4.496299   5.495917
 200   0.499944   1.500197   2.500091   3.500045   4.499644   5.501108
 400   0.500115   1.499999   2.500307   3.500226   4.499921   5.500079
 800   0.500094   1.500066   2.500056   3.500199   4.500287   5.499995

Autovectores guardados en data_oscilador_numerov.txt


In [20]:
import numpy as np
import matplotlib.pyplot as plt

def potencial_oscilador(x):
    return 0.5 * x**2

def numerov_propagar(E, x, h, V_func):
    """Propaga la función de onda usando la fórmula de Numerov"""
    n = len(x)
    psi = np.zeros(n)
    
    # k^2 = 2 * (E - V)
    k2 = 2.0 * (E - V_func(x))
    
    # Condiciones iniciales (para estados ligados, empezamos muy cerca de 0)
    psi[0] = 0.0
    psi[1] = 1e-5  # Un valor pequeño para iniciar la integración
    
    # Pre-calculamos el factor h^2 / 12
    h12 = (h**2) / 12.0
    
    # Iteración de Numerov (Tradicional)
    for i in range(1, n - 1):
        term_i   = 2.0 * (1.0 - 5.0 * h12 * k2[i]) * psi[i]
        term_im1 = (1.0 + h12 * k2[i-1]) * psi[i-1]
        den      = 1.0 + h12 * k2[i+1]
        psi[i+1] = (term_i - term_im1) / den
        
        # Guardián contra divergencias (si psi crece demasiado, cortamos)
        if abs(psi[i+1]) > 1e10: 
            return psi[i+1]
            
    return psi[-1] # Retorna el valor en el borde derecho para la bisección

def buscar_energias(V_func, x, h, E_min, E_max, dE=0.1):
    """Busca cambios de signo en el borde para localizar energías"""
    energias_propias = []
    e_puntos = np.arange(E_min, E_max, dE)
    
    # Evaluamos el valor final de psi para un rango de energías
    valores_borde = []
    for E in e_puntos:
        valores_borde.append(numerov_propagar(E, x, h, V_func))
    
    # Detectamos cruces por cero (cambios de signo)
    for i in range(len(valores_borde) - 1):
        if valores_borde[i] * valores_borde[i+1] < 0:
            # Refinamos con bisección simple
            e_low, e_high = e_puntos[i], e_puntos[i+1]
            for _ in range(20): # 20 pasos de bisección para precisión
                e_mid = (e_low + e_high) / 2
                if numerov_propagar(e_low, x, h, V_func) * numerov_propagar(e_mid, x, h, V_func) < 0:
                    e_high = e_mid
                else:
                    e_low = e_mid
            energias_propias.append((e_low + e_high) / 2)
            
    return energias_propias

# --- Configuración Principal ---
Rmin, Rmax = -6.0, 6.0  # Reducimos rango para el oscilador (más eficiente)
N = 1000
h = (Rmax - Rmin) / N
x = np.linspace(Rmin, Rmax, N)

print("Buscando los primeros autovalores del oscilador (Tradicional)...")
# El oscilador tiene energías 0.5, 1.5, 2.5...
autovalores = buscar_energias(potencial_oscilador, x, h, 0.1, 6.0)

print("\nEnergías encontradas:")
for i, E in enumerate(autovalores):
    print(f"Estado {i}: E = {E:.6f}")

Buscando los primeros autovalores del oscilador (Tradicional)...

Energías encontradas:
Estado 0: E = 0.500501
Estado 1: E = 1.501502
Estado 2: E = 2.502502
Estado 3: E = 3.503503
Estado 4: E = 4.504505
Estado 5: E = 5.505506


## Potencial |x|

In [3]:
import numpy as np

def potencial_valor_absoluto(x):
    """Define el potencial del sistema: V(x) = |x|"""
    return np.abs(x)

def propagacion_numerov(E, x, h, potencial_func):
    """
    Realiza la integración de la ecuación de Schrödinger paso a paso.
    Retorna el valor de la función de onda en el borde derecho.
    """
    n = len(x)
    psi = np.zeros(n)
    k2 = 2.0 * (E - potencial_func(x))
    
    # Condiciones iniciales en el extremo izquierdo
    psi[0] = 0.0
    psi[1] = 1e-7 
    h12 = (h**2) / 12.0
    
    for i in range(1, n - 1):
        # Fórmula de recurrencia de Numerov (4to orden)
        termino_i = 2.0 * (1.0 - 5.0 * h12 * k2[i]) * psi[i]
        termino_prev = (1.0 + h12 * k2[i-1]) * psi[i-1]
        denominador = 1.0 + h12 * k2[i+1]
        
        psi[i+1] = (termino_i - termino_prev) / denominador
        
        # Evitar overflow si la energía no es un autovalor (la función explota)
        if abs(psi[i+1]) > 1e10:
            return psi[i+1]
            
    return psi[-1]

def calcular_niveles_energia():
    # --- Parámetros de la simulación ---
    x_min, x_max = -10.0, 10.0
    n_minimo = 50      # N inicial para el estudio de convergencia
    n_maximo = 1600    # N final para la exportación de datos
    num_estados = 6    # Cantidad de autovalores a buscar
    
    print(f"Buscando los primeros {num_estados} niveles de energía propia:")
    print(f"{'N':>5} | {'E0':>8} {'E1':>8} {'E2':>8} {'E3':>8} {'E4':>8} {'E5':>8}")
    print("-" * 75)

    n_puntos = n_minimo
    while n_puntos <= n_maximo:
        x = np.linspace(x_min, x_max, n_puntos)
        h = (x_max - x_min) / (n_puntos - 1)
        
        # Escaneo de energía (Método de disparo)
        rango_energia = np.linspace(0.1, 10.0, 500)
        autovalores = []
        
        valor_borde_prev = propagacion_numerov(rango_energia[0], x, h, potencial_valor_absoluto)
        
        for i in range(1, len(rango_energia)):
            E_actual = rango_energia[i]
            valor_borde_actual = propagacion_numerov(E_actual, x, h, potencial_valor_absoluto)
            
            # Si hay un cambio de signo en el borde, refinamos el autovalor
            if valor_borde_prev * valor_borde_actual < 0:
                e_inferior, e_superior = rango_energia[i-1], rango_energia[i]
                
                # Bisección para alta precisión
                for _ in range(30):
                    e_medio = (e_inferior + e_superior) / 2
                    if propagacion_numerov(e_inferior, x, h, potencial_valor_absoluto) * \
                       propagacion_numerov(e_medio, x, h, potencial_valor_absoluto) < 0:
                        e_superior = e_medio
                    else:
                        e_inferior = e_medio
                autovalores.append((e_inferior + e_superior) / 2)
            
            valor_borde_prev = valor_borde_actual
            if len(autovalores) == num_estados:
                break

        # Imprimir fila de convergencia
        energias_str = " ".join([f"{E:8.5f}" for E in autovalores])
        print(f"{n_puntos:5d} | {energias_str}")

        # Guardar resultados finales al alcanzar la resolución máxima
        if n_puntos == n_maximo:
            probabilidades = np.zeros((n_puntos, num_estados))
            for j in range(num_estados):
                E = autovalores[j]
                psi = np.zeros(n_puntos)
                k2 = 2.0 * (E - potencial_valor_absoluto(x))
                psi[0], psi[1] = 0.0, 1e-7
                h12 = (h**2) / 12.0
                
                for i in range(1, n_puntos - 1):
                    psi[i+1] = (2.0*(1.0-5.0*h12*k2[i])*psi[i] - (1.0+h12*k2[i-1])*psi[i-1]) / (1.0+h12*k2[i+1])
                
                # Normalización de la densidad de probabilidad
                integral = np.sum(psi**2) * h
                probabilidades[:, j] = psi**2 / integral

            # Escritura del archivo de salida
            with open("data_abs_numerov.txt", "w") as f:
                header_energies = " ".join([f"{E:10.6f}" for E in autovalores])
                f.write(f"#{n_puntos//2:4d}      {header_energies}\n")
                for i in range(n_puntos):
                    columnas = " ".join([f"{probabilidades[i, j]:10.5f}" for j in range(num_estados)])
                    f.write(f"{x[i]:10.5f} {columnas}\n")
            
            print(f"\nSimulación finalizada. Datos guardados en 'data_abs_numerov.txt'")

        # Duplicar la resolución para la siguiente iteración
        n_puntos *= 2

if __name__ == "__main__":
    calcular_niveles_energia()

Buscando los primeros 6 niveles de energía propia:
    N |       E0       E1       E2       E3       E4       E5
---------------------------------------------------------------------------
   50 |  0.81731  1.85472  2.57986  3.24160  3.82394  4.37509
  100 |  0.81073  1.85570  2.57870  3.24444  3.82595  4.38130
  200 |  0.80914  1.85575  2.57826  3.24460  3.82581  4.38165
  400 |  0.80875  1.85576  2.57814  3.24461  3.82574  4.38167
  800 |  0.80865  1.85576  2.57811  3.24461  3.82572  4.38167
 1600 |  0.80862  1.85576  2.57810  3.24461  3.82572  4.38167

Simulación finalizada. Datos guardados en 'data_abs_numerov.txt'


## Potencial x⁴

In [4]:
import numpy as np

def potencial_cuartico(x):
    return x**4

def propagacion_numerov(E, x, h, potencial_func):
    n = len(x)
    psi = np.zeros(n)
    k2 = 2.0 * (E - potencial_func(x))
    
    psi[0] = 0.0
    psi[1] = 1e-8 
    h12 = (h**2) / 12.0
    
    for i in range(1, n - 1):
        f_i = 1.0 + h12 * k2[i+1]
        termino_i = 2.0 * (1.0 - 5.0 * h12 * k2[i]) * psi[i]
        termino_prev = (1.0 + h12 * k2[i-1]) * psi[i-1]
        
        psi[i+1] = (termino_i - termino_prev) / f_i
        
        # Guardián más permisivo para evitar cortes falsos
        if abs(psi[i+1]) > 1e12:
            return psi[i+1]
            
    return psi[-1]

def calcular_niveles_cuarticos():
    # AJUSTE CRÍTICO: Rango de x más pequeño para evitar que la función se pierda
    x_min, x_max = -3.0, 3.0  
    n_minimo = 50      
    n_maximo = 1600    
    num_estados = 6    
    
    print(f"Estudio de Convergencia: Oscilador Cuártico V(x) = x^4")
    print(f"{'N':>5} | {'E0':>8} {'E1':>8} {'E2':>8} {'E3':>8} {'E4':>8} {'E5':>8}")
    print("-" * 80)

    n_puntos = n_minimo
    while n_puntos <= n_maximo:
        x = np.linspace(x_min, x_max, n_puntos)
        h = (x_max - x_min) / (n_puntos - 1)
        
        # Rango de E suficiente para encontrar los 6 niveles
        rango_energia = np.linspace(0.1, 25.0, 1500)
        autovalores = []
        
        f_prev = propagacion_numerov(rango_energia[0], x, h, potencial_cuartico)
        
        for i in range(1, len(rango_energia)):
            E_actual = rango_energia[i]
            f_actual = propagacion_numerov(E_actual, x, h, potencial_cuartico)
            
            if f_prev * f_actual < 0:
                e_low, e_high = rango_energia[i-1], rango_energia[i]
                for _ in range(35): # Más pasos de bisección para precisión
                    em = (e_low + e_high) / 2
                    if propagacion_numerov(e_low, x, h, potencial_cuartico) * \
                       propagacion_numerov(em, x, h, potencial_cuartico) < 0:
                        e_high = em
                    else:
                        e_low = em
                autovalores.append((e_low + e_high) / 2)
            
            f_prev = f_actual
            if len(autovalores) == num_estados:
                break

        if len(autovalores) < num_estados:
            print(f"{n_puntos:5d} | Error: Solo se hallaron {len(autovalores)} estados.")
        else:
            energias_str = " ".join([f"{E:8.4f}" for E in autovalores])
            print(f"{n_puntos:5d} | {energias_str}")

        # --- BLOQUE NUEVO: GUARDAR DATOS ---
        if n_puntos == n_maximo and len(autovalores) == num_estados:
            densidades = np.zeros((n_puntos, num_estados))
            
            for j in range(num_estados):
                E = autovalores[j]
                psi = np.zeros(n_puntos)
                k2 = 2.0 * (E - potencial_cuartico(x))
                psi[0], psi[1] = 0.0, 1e-8
                h12 = (h**2) / 12.0
                
                # Re-propagar para obtener la función de onda completa
                for i in range(1, n_puntos - 1):
                    psi[i+1] = (2.0*(1.0-5.0*h12*k2[i])*psi[i] - (1.0+h12*k2[i-1])*psi[i-1]) / (1.0+h12*k2[i+1])
                
                # Normalización: Integral de psi^2 dx = 1
                norma = np.sum(psi**2) * h
                densidades[:, j] = psi**2 / norma

            # Escritura al archivo
            with open("data_x4_numerov.txt", "w") as f:
                header_E = " ".join([f"{E:10.6f}" for E in autovalores])
                f.write(f"#{n_puntos//2:4d}      {header_E}\n")
                for i in range(n_puntos):
                    linea_densidades = " ".join([f"{densidades[i, j]:10.5f}" for j in range(num_estados)])
                    f.write(f"{x[i]:10.5f} {linea_densidades}\n")
            
            print(f"\n[OK] Datos de densidad guardados en 'data_x4_numerov.txt'")
        # -----------------------------------

        n_puntos *= 2

if __name__ == "__main__":
    calcular_niveles_cuarticos()

Estudio de Convergencia: Oscilador Cuártico V(x) = x^4
    N |       E0       E1       E2       E3       E4       E5
--------------------------------------------------------------------------------
   50 |   0.6680   2.3936   4.6965   7.3348  10.2420  13.3743
  100 |   0.6680   2.3936   4.6968   7.3357  10.2442  13.3790
  200 |   0.6680   2.3936   4.6968   7.3357  10.2443  13.3793
  400 |   0.6680   2.3936   4.6968   7.3357  10.2443  13.3793
  800 |   0.6680   2.3936   4.6968   7.3357  10.2443  13.3793
 1600 |   0.6680   2.3936   4.6968   7.3357  10.2443  13.3793

[OK] Datos de densidad guardados en 'data_x4_numerov.txt'


## Doble Pozo Potencial V(x)= x⁴ - 5x²

In [7]:
import numpy as np

def potencial_doble_pozo(x, A_param=5.0):
    """Define el potencial de doble pozo: V(x) = x^4 - A*x^2"""
    return x**4 - A_param * x**2

def propagacion_numerov(E, x, h, potencial_func, A_param):
    """
    Realiza la integración de la ecuación de Schrödinger paso a paso.
    Retorna el valor de la función de onda en el borde derecho.
    """
    n = len(x)
    psi = np.zeros(n)
    # k2 = 2 * (E - V)
    k2 = 2.0 * (E - potencial_func(x, A_param))
    
    # Condiciones iniciales
    psi[0] = 0.0
    psi[1] = 1e-8 
    h12 = (h**2) / 12.0
    
    for i in range(1, n - 1):
        f_i = 1.0 + h12 * k2[i+1]
        termino_i = 2.0 * (1.0 - 5.0 * h12 * k2[i]) * psi[i]
        termino_prev = (1.0 + h12 * k2[i-1]) * psi[i-1]
        
        psi[i+1] = (termino_i - termino_prev) / f_i
        
        # Guardián contra divergencias numéricas
        if abs(psi[i+1]) > 1e12:
            return psi[i+1]
            
    return psi[-1]

def calcular_niveles_doble_pozo():
    # --- Parámetros de la simulación ---
    x_min, x_max = -4.0, 4.0  # Rango optimizado para este potencial
    n_minimo = 50      
    n_maximo = 1600    
    num_estados = 6    
    A_val = 5.0               # Parámetro A que define la profundidad de los pozos
    
    print(f"Estudio de Convergencia: Doble Pozo V(x) = x^4 - {A_val}x^2")
    print(f"{'N':>5} | {'E0':>8} {'E1':>8} {'E2':>8} {'E3':>8} {'E4':>8} {'E5':>8}")
    print("-" * 80)

    n_puntos = n_minimo
    while n_puntos <= n_maximo:
        x = np.linspace(x_min, x_max, n_puntos)
        h = (x_max - x_min) / (n_puntos - 1)
        
        # Rango de energía: el doble pozo con A=5 tiene su mínimo en V = -6.25
        # Por lo tanto, buscamos desde -7.0 hasta 15.0
        rango_energia = np.linspace(-7.0, 15.0, 1200)
        autovalores = []
        
        f_prev = propagacion_numerov(rango_energia[0], x, h, potencial_doble_pozo, A_val)
        
        for i in range(1, len(rango_energia)):
            E_actual = rango_energia[i]
            f_actual = propagacion_numerov(E_actual, x, h, potencial_doble_pozo, A_val)
            
            if f_prev * f_actual < 0:
                e_low, e_high = rango_energia[i-1], rango_energia[i]
                # Bisección para refinar el autovalor
                for _ in range(35):
                    em = (e_low + e_high) / 2
                    if propagacion_numerov(e_low, x, h, potencial_doble_pozo, A_val) * \
                       propagacion_numerov(em, x, h, potencial_doble_pozo, A_val) < 0:
                        e_high = em
                    else:
                        e_low = em
                autovalores.append((e_low + e_high) / 2)
            
            f_prev = f_actual
            if len(autovalores) == num_estados:
                break

        # Verificación y salida en consola
        if len(autovalores) < num_estados:
            print(f"{n_puntos:5d} | Error: Solo se hallaron {len(autovalores)} estados.")
        else:
            energias_str = " ".join([f"{E:8.4f}" for E in autovalores])
            print(f"{n_puntos:5d} | {energias_str}")

        # Guardar resultados finales en la resolución máxima
        if n_puntos == n_maximo:
            probabilidades = np.zeros((n_puntos, num_estados))
            for j in range(num_estados):
                E = autovalores[j]
                psi = np.zeros(n_puntos)
                k2 = 2.0 * (E - potencial_doble_pozo(x, A_val))
                psi[0], psi[1] = 0.0, 1e-8
                h12 = (h**2) / 12.0
                for i in range(1, n_puntos - 1):
                    psi[i+1] = (2.0*(1.0-5.0*h12*k2[i])*psi[i] - (1.0+h12*k2[i-1])*psi[i-1]) / (1.0+h12*k2[i+1])
                
                integral = np.sum(psi**2) * h
                probabilidades[:, j] = psi**2 / integral

            with open("data_doble_pozo_numerov.txt", "w") as f:
                header_energies = " ".join([f"{E:10.6f}" for E in autovalores])
                f.write(f"#{n_puntos//2:4d}      {header_energies}\n")
                for i in range(n_puntos):
                    columnas = " ".join([f"{probabilidades[i, j]:10.5f}" for j in range(num_estados)])
                    f.write(f"{x[i]:10.5f} {columnas}\n")
            
            print(f"\n[OK] Datos guardados en 'data_doble_pozo_numerov.txt'")

        n_puntos *= 2

if __name__ == "__main__":
    calcular_niveles_doble_pozo()

Estudio de Convergencia: Doble Pozo V(x) = x^4 - 5.0x^2
    N |       E0       E1       E2       E3       E4       E5
--------------------------------------------------------------------------------
   50 |  -0.7574  -0.2118   1.9191   3.8322   6.1703   8.7210
  100 |  -4.1358  -4.1191  -0.7568  -0.2107   1.9212   3.8370
  200 |  -4.1358  -4.1191  -0.7568  -0.2107   1.9213   3.8373
  400 |  -4.1358  -4.1191  -0.7568  -0.2107   1.9213   3.8373
  800 |  -4.1358  -4.1191  -0.7568  -0.2107   1.9213   3.8373
 1600 |  -4.1358  -4.1191  -0.7568  -0.2107   1.9213   3.8373

[OK] Datos guardados en 'data_doble_pozo_numerov.txt'
